In [ ]:
import os
import sys
os.getcwd()

In [ ]:
# importing libraries & making torch.device object for GPU

# neural network packages
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch import Tensor
from torch.utils.data import DataLoader
sys.path.append('..\\..\\PHYS417_Project')
from nnrunner import NetRunner

# data packages
import numpy as np
import math
import pandas as pd
import sklearn.preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import fndict as fd
import random
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import StandardScaler

# visual packages
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tqdm.notebook as tq
from tqdm.notebook import trange
import warnings

# Create a torch.device object to tell pytorch where to store your tensors: cpu or gpu
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')